# How to use Beautiful soup to scrape the table from website


This notebook will show you how to scrape postal codes of Canada from the wikipedia page as follow.  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Step 1. Install necessary library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs

from lxml import etree
import requests
print("libraries installed")

libraries installed


### Step 2. Scrape the html text

In [28]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text # get the html text by requests
soup = bs(url,'lxml') # transform html text into beatiful soup object


### Step 3. Match the target table in html text

In [29]:
match = soup.table  # Note that ".table" will show the first table in the html text.  In this case, it is our target table.
# pinrt(match)

In [30]:
table =soup.find_all('table', class_="wikitable sortable")  # find all tables in the html which class is "wikitable sortable", it will return a list
print('There are {} tables in this url '.format(len(table))) # Use len() to count numbers of table in this list, In this case, only return one table

# you can use ".find" to find the atrribute of your target, in this case is table, and class that it is.
# class is a python function. so to make a difference, we have to add underscore affter "class"  

There are 1 tables in this url 


### Step 4. Retrieve the data and save as pandas dataframe (Raw Data of Table)

In [31]:
# little test for retrieve the table data 
    
soup_cont = table[0].find_all('tr') # soup_cont = soup contents. Use "tr" to cut data 

print(soup_cont[1]) # This is the first data of the table 
clean = soup_cont[1].find_all('td') # use "td" to get the data string

Postalcode = clean[0].text.strip("\n")   # use ".text" to clean the html code and strip() to remove "\n"
Borough = clean[1].text.strip("\n")
Neighborhood = clean[2].text.strip("\n")

print('This is the first row of data :',[Postalcode,Borough,Neighborhood])



<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
This is the first row of data : ['M1A', 'Not assigned', 'Not assigned']


In [32]:
# Create a dictionary to insert the data of table
d = {'PostalCode':[],'Borough':[],'Neighborhood':[]} 

# Use loop to get all of the data
for i in range(len(soup_cont)-1):  
    d['PostalCode'].append(soup_cont[i+1].find_all('td')[0].text.strip("\n"))
    d['Borough'].append(soup_cont[i+1].find_all('td')[1].text.strip("\n"))
    d['Neighborhood'].append(soup_cont[i+1].find_all('td')[2].text.strip("\n"))

# Transform the python dictionary to pandas dataframe
target_table = pd.DataFrame(data=d) 

# Exchange the columns location to meet the requirement
cols = ['PostalCode','Borough','Neighborhood']
target_table = target_table.loc[:,cols]

# This is unclear row data
target_table.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 5. Clean the raw data to meet the requirement

- Drop all the rows for those area that is **not assigned** a **Borough**

In [74]:
clean_df = target_table[target_table.Borough != 'Not assigned']
clean_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


- Assign **Borough** to **Neighborhood** for those area that is **not assigned** a **Neighborhood**

In [75]:
# Find that area with "Not assigned" Neiborhood
clean_df.loc[clean_df.Neighborhood == 'Not assigned']


,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [76]:
# Replace Neighborhood with its Borough
clean_df[clean_df.Neighborhood == 'Not assigned'].index[0] #　Find target index
clean_df.loc[clean_df[clean_df.Neighborhood == 'Not assigned'].index[0],'Neighborhood'] = 'Queen\'s Park'
clean_df.head(10)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### Step 6. Group the data based on its postalcode

In [107]:
data = clean_df
group_data = clean_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "[%s]" % ', '.join(x)) # use ".groupby" and ".apply" method to save the group data 
group_data.reset_index().head(12)  # This is an important process to flatten the tuple.

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [109]:
group_data.reset_index().shape[0]

103